# Constructive heuristics comparison

The objective of the $\alpha$-neighbor $p$-center problem can be thought of as distributing the facilities among the clients to cover them efficiently, which is the actual goal of the $p$-center problem, so a constructive heuristic that uses its objective function will be tested and compared against a greedy heuristic that takes into account the objective function of this problem.

There will be used 20 random instances of size $n = 50$, $p = 5$ and other 20 of size $n = 400$, $p = 20$, and each one will be tested with both $\alpha = 2$ and $\alpha = 3$. The coordinates of the points are between 0 and 1000 for both planes.

In [2]:
from copy import deepcopy
from typing import List

from models.instance import Instance


def generate_instances(amount: int, n: int, p: int) -> List[Instance]:
    alpha2 = [
        Instance.random(n, p, 2, 1000, 1000)
        for _ in range(amount)
    ]
    alpha3 = deepcopy(alpha2)
    for i in alpha3:
        i.alpha = 3
    return alpha2 + alpha3

In [3]:
instances = generate_instances(20, 50, 5) + generate_instances(20, 400, 20)

We will use the following code to measure the time taken by the evaluations and the objective function results, formatted in a Pandas DataFrame.

In [2]:
import timeit

import pandas as pd

from heuristics.constructive import pdp_based, greedy
from utils import eval_obj_func

def measure(instance, heuristic):
    start = timeit.default_timer()
    solution = heuristic(instance)
    time = timeit.default_timer() - start
    of = eval_obj_func(instance, solution)
    return heuristic.__name__, solution, of, time

def get_dataframe(data):
    return pd.DataFrame({
        colname: [d[i] for d in data]
        for colname, i in zip(
            ('n', 'p', 'a', 'heuristic', 'solution', 'OF', 'seconds'),
            range(len(data[0])))
    })

## Comparing data

In [7]:
import os

OUT_FOLDER = 'nb_results\\constructive'

filepath = os.path.join(OUT_FOLDER, 'pdp_df.csv')
if os.path.exists(filepath):
    pdp_df = pd.read_csv(filepath)
else:
    pdp_data = [(*i.get_parameters(), *measure(i, pdp_based))  for i in instances]
    pdp_df = get_dataframe(pdp_data)
    pdp_df.to_csv(filepath, index=False)
pdp_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,pdp_based,"{33, 5, 37, 40, 28}",628,0.000760
1,50,5,2,pdp_based,"{33, 2, 34, 13, 14}",607,0.000535
2,50,5,2,pdp_based,"{35, 5, 8, 9, 42}",794,0.000527
3,50,5,2,pdp_based,"{18, 19, 13, 45, 14}",611,0.000617
4,50,5,2,pdp_based,"{1, 7, 27, 29, 47}",643,0.000525
...,...,...,...,...,...,...,...
75,400,20,3,pdp_based,"{66, 68, 198, 74, 395, 397, 399, 16, 81, 209, ...",406,0.074006
76,400,20,3,pdp_based,"{261, 8, 201, 202, 13, 22, 281, 155, 222, 289,...",390,0.093924
77,400,20,3,pdp_based,"{1, 259, 325, 198, 134, 201, 204, 77, 78, 208,...",375,0.073400
78,400,20,3,pdp_based,"{391, 10, 13, 77, 271, 16, 17, 274, 144, 84, 2...",372,0.064396


Saving the evaluation to a CSV:

In [17]:
filepath = os.path.join(OUT_FOLDER, 'greedy_df.csv')
if os.path.exists(filepath):
    greedy_df = pd.read_csv(filepath)
else:
    greedy_data = [(*i.get_parameters(), *measure(i, greedy))  for i in instances]
    greedy_df = get_dataframe(greedy_data)
    greedy_df.to_csv(filepath, index=False)
greedy_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,greedy,"{2, 3, 37, 40, 28}",600,0.216955
1,50,5,2,greedy,"{33, 1, 13, 14, 15}",607,0.067130
2,50,5,2,greedy,"{1, 35, 8, 44, 31}",627,0.021097
3,50,5,2,greedy,"{18, 3, 4, 43, 13}",591,0.050969
4,50,5,2,greedy,"{20, 38, 7, 9, 29}",610,0.020568
...,...,...,...,...,...,...,...
75,400,20,3,greedy,"{1, 2, 3, 4, 5, 198, 6, 8, 7, 9, 10, 14, 16, 2...",369,32.796715
76,400,20,3,greedy,"{1, 2, 3, 4, 5, 6, 7, 8, 201, 10, 12, 14, 16, ...",401,36.384875
77,400,20,3,greedy,"{1, 2, 3, 67, 4, 6, 201, 9, 11, 12, 77, 78, 14...",434,33.087493
78,400,20,3,greedy,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 138, 10, 12, 13, 1...",404,29.960488


Calculating statistics of the results:

In [23]:
filtered_data = {
    heuristic: {
        f'n{n}': {
            f'a{alpha}': df[
                (df['n'] == n) &
                (df['a'] == alpha)
                ].iloc[:, [0, 1, 2, 3, 5, 6]]
            for alpha in (2, 3)
        }
        for n in (50, 400)
    }
    for heuristic, df in (('pdp', pdp_df), ('greedy', greedy_df))
}

In [291]:
stats = (filtered_data['pdp']['n400']['a2']
    .compare(filtered_data['greedy']['n400']['a2'], keep_equal=True)
    .rename(columns={ 'self': 'pdp', 'other': 'greedy' })
    .drop(columns='heuristic'))

stats['OF', 'absolute'] = stats['OF', 'pdp'] - stats['OF', 'greedy']
stats['OF', 'relative'] = (stats['OF', 'absolute'] / stats['OF', 'pdp'])

order = ['pdp', 'greedy', 'absolute', 'relative']
stats = stats.loc[:, (('OF', 'seconds'), order)]

stats[stats['OF', 'absolute'] > 0]

def calc_stats(n, a):
    ncol = f'n{n}'
    acol = f'a{a}'
    stats = (filtered_data['pdp'][ncol][acol]
        .compare(filtered_data['greedy'][ncol][acol], keep_equal=True)
        .rename(columns={ 'self': 'pdp', 'other': 'greedy' })
        .drop(columns='heuristic'))

    stats['OF', 'absolute'] = stats['OF', 'pdp'] - stats['OF', 'greedy']
    stats['OF', '%'] = (stats['OF', 'absolute'] / stats['OF', 'pdp']) * 100

    order = ['pdp', 'greedy', 'absolute', '%']
    tops = ('OF', 'seconds')
    stats = stats.loc[:, (tops, order)]

    winnings = [
        stats[stats['OF', 'absolute'] <= 0].count()[0],
        stats[stats['OF', 'absolute'] > 0].count()[0],
        '', '', '', ''
    ]
    average = [
        stats[col].mean()
        for col in stats.columns
    ]

    stats.loc['winnings'] = winnings
    stats.loc['average'] = average

    return stats

s = calc_stats(400, 2)
s

d:\NETO\programacion\alpha-neighbor-p-center-problem\venv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
d:\NETO\programacion\alpha-neighbor-p-center-problem\venv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


OF                              seconds           
            pdp greedy absolute          %       pdp     greedy
40        307.0  304.0        3   0.977199   0.07164   45.30798
41        292.0  350.0      -58 -19.863014  0.102789  42.235033
42        310.0  309.0        1   0.322581   0.12521  50.214393
43        294.0  426.0     -132 -44.897959  0.051584  45.909697
44        317.0  297.0       20   6.309148  0.057793  48.009938
45        297.0  358.0      -61 -20.538721  0.049126  51.167755
46        328.0  336.0       -8  -2.439024   0.05111  45.846795
47        323.0  354.0      -31  -9.597523  0.050484  45.359293
48        275.0  362.0      -87 -31.636364  0.052707  49.758392
49        339.0  341.0       -2  -0.589971  0.062176  50.711559
50        304.0  277.0       27   8.881579   0.05425  48.212123
51        324.0  362.0      -38 -11.728395   0.05771  51.083328
52        294.0  315.0      -21  -7.142857  0.053468  50.279512
53        312.0  336.0      -24  -7.692308  0.056241   58.75892
54        276.0  308.0      -32 -11.594203   0.05742  55.484057
55        348.0  302.0       46  13.218391  0.053719  52.859262
56        359.0  299.0       60  16.713092  0.057072  44.194823
57        343.0  348.0       -5  -1.457726  0.056023  50.252007
58        276.0  348.0      -72 -26.086957  0.058349  38.374765
59        284.0  270.0       14   4.929577  0.058375  43.111878
winnings   13.0    7.0                                         
average   310.1  330.1    -20.0  -7.195673  0.061862  48.356575